In [1]:
import os, sys
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
#------------------------------------load directory and clean useless files--------------------------------------------------------------------
def getFilesFromDir (path):
    dir_content = os.listdir(path)
    dir_clean = filter(lambda x: (".DS_Store" not in x) and ("cmds" not in x), dir_content)
    return dir_clean
#file path
base_path = "/Users/yuliang/Downloads/naive_bayes"
ham_path = base_path + "/easy_ham"
ham_path2 = base_path + "/easy_ham_2"
spam_path = base_path + "/spam"
spam_path2 = base_path + "/spam_2"
#get files from directory
file_list_ham = getFilesFromDir(ham_path)
file_list_ham2 = getFilesFromDir(ham_path2)
file_list_spam = getFilesFromDir(spam_path)
file_list_spam2 = getFilesFromDir(spam_path2)
#------------------------------------load data in each file in the directory---------------------------------------------
def getMessage (file_list, path, amount_of_samples):
    strip_html = False
    str_list = ["" for x in range (amount_of_samples)]
    i = 0
#open files in the directory
    for index in range (amount_of_samples):
        fo = open (path + '/' + file_list[index])
        lines = fo.readlines()
        fo.close()

        for index in range (len(lines)):
            if (index <= (len(lines)-2)):
                if (('\n' in lines[index]) and (lines[index + 1] == '\n')):
                    lines[index:index+2] = [''.join(lines[index:index+2])]
#move to the first line break of the email which indicates the message body
        start_position = next(x for x in lines if '\n\n' in x)
        start_position_index = lines.index(start_position)
        lines_clean = lines[start_position_index + 1: len(lines)]
#convert list into string
        str1 = ''.join(lines_clean)
#only keep words and spaces in the string
        str2 = re.sub(r'([^\s\w]|_)+', '', str1)
        str_list[i] = str2
        i = i + 1     
    return str_list
#set amount of samples to take, 500 samples equals to the number of spam samples
amountOfSamplesPerSet = 500
amountOfFeaturesToTake = 50 #features can be 50, 100, 200, 400...
amountOfSamplesPerSet_test = 1390 #amount of spam and ham to take for testing

str_clean_ham = getMessage (file_list_ham, ham_path, amountOfSamplesPerSet)
str_clean_ham2 = getMessage (file_list_ham2, ham_path2, amountOfSamplesPerSet_test)
str_clean_spam = getMessage (file_list_spam, spam_path, amountOfSamplesPerSet)
str_clean_spam2 = getMessage (file_list_spam2, spam_path2, amountOfSamplesPerSet_test)

email_words_list = str_clean_ham + str_clean_spam
#------------------------------------get stop words--------------------------------------------------------------------
def getStopWords (path):
    fo = open (path)
    lines = fo.readlines()
    lines_clean = map(lambda x: str.replace(x, '\n', ''), lines)
    fo.close()
    return lines_clean
stop_words_given = getStopWords ("/Users/yuliang/Downloads/naive_bayes/stopwords.txt")
#-----------------------------------Term Document Matrix---------------------------------------------------------------
#establish TDM using sklearn.feature_extraction.text.CountVectorizer
#get ham top features
vector = CountVectorizer(stop_words = stop_words_given,
                         analyzer='word',
                         decode_error = 'ignore',
                         max_features = amountOfFeaturesToTake)
words_counts = vector.fit_transform(str_clean_ham)
words_counts = words_counts.toarray()
feature_ham = vector.get_feature_names()

#get spam top features
vector2 = CountVectorizer(stop_words = stop_words_given,
                          analyzer='word',
                          decode_error = 'ignore',
                          max_features = amountOfFeaturesToTake)

words_counts2 = vector2.fit_transform(str_clean_spam)
words_counts2 = words_counts2.toarray()
feature_spam = vector2.get_feature_names()

#ham top features + spam top features
feature_email_raw = feature_ham + feature_spam
feature_email_clean = set(feature_email_raw)
feature_email = list(feature_email_clean)

#final TDM
vector3 = CountVectorizer(decode_error = 'ignore',
                          vocabulary = feature_email)

words_counts3 = vector3.fit_transform(email_words_list)
words_counts3 = words_counts3.toarray()

#---------------------------------------classification: training and testing-------------------------------------------
#establish testing labels
y1 = np.zeros(amountOfSamplesPerSet_test)
y2 = np.ones(amountOfSamplesPerSet_test)
#establish training labels
y = np.zeros(2*amountOfSamplesPerSet)
for index in range(amountOfSamplesPerSet, len(y)):
    y[index] = 1
#training: Multinomial Naive Bayes
clf = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True).fit(words_counts3, y)
#ham test
new_counts = vector3.transform(str_clean_ham2)
#predication
predicted = clf.predict(new_counts)
#print(predicted)
#accuracy score
accu_score = clf.score(new_counts, y1)
print (accu_score)

#spam test
new_counts = vector3.transform(str_clean_spam2)
#predication
predicted = clf.predict(new_counts)
#print(predicted)
#accuracy score
accu_score = clf.score(new_counts, y2)
print (accu_score)
    
    

0.973381294964
0.842446043165


Run the code several times and change amount of features to take, we get the following results:
###Ham Result | Amount of Features to take | Ham (Correct) | Spam | | ----------------------------------|:-----------------------:| -----:| | 50 | 95.90% | 4.10% | | 100 | 97.34% | 2.66% | | 200 | 98.06% | 1.94% | | 400 | 98.49% | 1.51% |
###Spam Result | Amount of Features to take | Ham (Correct) | Spam | | ----------------------------------|:-----------------------:| -----:| | 50 | 75.32% | 24.68%| | 100 | 84.24% | 15.76%| | 200 | 86.04% | 13.96%| | 400 | 90.29% | 9.71% |